In [0]:
#storage test
#import os, uuid
import pandas as pd
from io import StringIO
from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from io import BytesIO
import os



In [0]:
%run "./password_manager"

In [0]:


connect_str = get_password('storage_conn_str')

In [0]:


#ashwin.8089@gmail.com

def upload_to_azure(connect_str, container_name, file_path,blob_name):
    try:
        blob_service_client = BlobServiceClient.from_connection_string(connect_str)
        container_client = blob_service_client.get_container_client(container_name)
        blob_client = container_client.get_blob_client(os.path.basename(file_path))
        with open(file_path, "rb") as data:
            blob_client.upload_blob(data)
        print(f"File {file_path} uploaded to Azure Blob Storage successfully.")
    except Exception as e:
        print(f"An error occurred: {e}")



def delete_blob_from_azure(connect_str, container_name, blob_name):
    try:
        blob_service_client = BlobServiceClient.from_connection_string(connect_str)
        container_client = blob_service_client.get_container_client(container_name)
        blob_client = container_client.get_blob_client(blob_name)
        #print(blob_client.exists())
        if(blob_client.exists()):
          blob_client.delete_blob()
          print(f"Blob '{blob_name}' deleted from container '{container_name}' successfully.")
        else:
            print(f"{blob_name}' does not exist in container '{container_name}")
    except Exception as e:
        print(f"An error occurred: {e}")


def get_container_list_storage(connect_str,starter):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    containers_list = blob_service_client.list_containers()
    containers = [container.name for container in containers_list if container.name.startswith(starter)]
    return containers


def get_blob_using_container_url(container_url,blob_name):
    container = ContainerClient.from_container_url(container_url) 

    blob_client = container.get_blob_client(blob_name)   

    blob_data = blob_client.download_blob()

    blob_data2 = blob_data.readall()    

    data = BytesIO(blob_data2)
    df = pd.read_excel(data, engine='openpyxl')
    return df


def get_blobs_list_container(connect_str,container_name,dir):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str) 

    container_client = blob_service_client.get_container_client(container_name)
    if(dir!=''):
       blob_list = container_client.list_blobs(name_starts_with=dir)
    else:
       blob_list = container_client.list_blobs()

    list_blobs =[]

    for blob in blob_list:
        list_blobs.append(blob.name)
    
    return list_blobs



def rename_blob( container_name, old_blob_name, new_blob_name,connect_storage_str=connect_str):
    blob_service_client =  BlobServiceClient.from_connection_string(connect_storage_str) 
    container_client = blob_service_client.get_container_client(container_name)
 
    old_blob_client = container_client.get_blob_client(old_blob_name)
    new_blob_client = container_client.get_blob_client(new_blob_name)
    new_blob_client.start_copy_from_url(old_blob_client.url)
    properties = new_blob_client.get_blob_properties()
    while properties.copy.status != 'success':
        properties = new_blob_client.get_blob_properties()
    old_blob_client.delete_blob()


def get_file_from_container(container_name,blob_name,connect_storage_str=connect_str):
    blob_service_client = BlobServiceClient.from_connection_string(connect_storage_str) 

    container_client = blob_service_client.get_container_client(container_name)

    blob_client = container_client.get_blob_client(blob_name)

    blob_data = blob_client.download_blob()
    
    blob_data2 = blob_data.readall()
    
    data = BytesIO(blob_data2)
    df = pd.read_excel(data, engine='openpyxl')
    return df
def get_file_data_from_container(connect_str,container_name,blob_name):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str) 

    container_client = blob_service_client.get_container_client(container_name)

    blob_client = container_client.get_blob_client(blob_name)

    blob_data = blob_client.download_blob()
    
    blob_data2 = blob_data.readall()
    
    data = BytesIO(blob_data2)
    return data

def get_filedata_from_container(connect_str,container_name,blob_name):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str) 

    container_client = blob_service_client.get_container_client(container_name)

    blob_client = container_client.get_blob_client(blob_name)

    blob_data = blob_client.download_blob()
    
    blob_data2 = blob_data.readall()
    
    data = BytesIO(blob_data2)
    return data




def write_to_storage(df, container_name, blob_name,connect_storage_str=connect_str):
    blob_service_client = BlobServiceClient.from_connection_string(connect_storage_str)
    container_client = blob_service_client.get_container_client(container_name)
    blob_client = container_client.get_blob_client(blob_name)
    output = BytesIO()
    df.to_csv(output, index=False)
    #df.to_csv(output, **params)
    output.seek(0)
    blob_client.upload_blob(output, overwrite=True)
    
    print("DataFrame successfully written to Azure Blob Storage.")


def write_to_storage_params(output, container_name, blob_name,connect_storage_str=connect_str):
    blob_service_client = BlobServiceClient.from_connection_string(connect_storage_str)
    container_client = blob_service_client.get_container_client(container_name)
    blob_client = container_client.get_blob_client(blob_name)
    #output = BytesIO()
    ##df.to_csv(output, index=False)
    #df.to_csv(output, **params)
    #output.seek(0)
    blob_client.upload_blob(output, overwrite=True)
    
    print("DataFrame successfully written to Azure Blob Storage.")




def ftp_file_from_azure_blob_to_server( container_name, blob_name,connect_storage_str=connect_str):
    blob_service_client = BlobServiceClient.from_connection_string(connect_storage_str)
    blob_client = blob_service_client.get_blob_client(container_name, blob_name)
    local_file_path = "./data/" + blob_name
    with open(local_file_path, "wb") as download_file:
        download_file.write(blob_client.download_blob().readall())



def copy_blob(source_container_name,source_blob_name, destination_container_name, destination_blob_name, connect_storage_str):
    blob_service_client = BlobServiceClient.from_connection_string(connect_storage_str)
    source_blob_client = blob_service_client.get_blob_client(container=source_container_name, blob=source_blob_name)
    source_blob_url = source_blob_client.url
    destination_blob_client = blob_service_client.get_blob_client(container=destination_container_name, blob=destination_blob_name)
    copy_operation = destination_blob_client.start_copy_from_url(source_blob_url)
    return copy_operation['copy_status']


